# Sklearn Pipeline for Scoring New Data

In [1]:
import pickle
import pandas as pd

# Import Data

In [2]:
# load the local dataset 
df = pd.read_csv(
        '../data/mal_customers_dataset.csv',
        index_col='CustomerID')

# rename column name
df.rename(
        columns={
                'Genre': 'Gender',
                'Annual_Income_(k$)': 'Income'}, 
        inplace=True)

df.head()

,Gender,Age,Income,Spending_Score
CustomerID,,,,
1,Male,19,15,39
2,Male,21,15,81
3,Female,20,16,6
4,Female,23,16,77
5,Female,31,17,40


# Load Model Artifacts

In [3]:
with open('../artifacts/numeric_imputer.pickle', 'rb') as filename: # trained model to impute missing numeric data
    numeric_imputer = pickle.load(filename)

with open('../artifacts/categorical_imputer.pickle', 'rb') as filename: # trained model to impute missing categorical data
    categorical_imputer = pickle.load(filename) 

with open('../artifacts/rare_encoder.pickle', 'rb') as filename: # trained model to encode rare labels
    rare_encoder = pickle.load(filename)

with open('../artifacts/capper.pickle', 'rb') as filename: # trained model to cap outliers
    capper = pickle.load(filename)   

with open('../artifacts/enc.pickle', 'rb') as filename: # trained one hot encoder
    enc = pickle.load(filename)

with open('../artifacts/model.pickle', 'rb') as filename: # trained random forrest classifier
    model = pickle.load(filename)

# Transform Dataset

In [4]:
# get numeric and categorical columns
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_columns = df.select_dtypes(include=numerics).columns.to_list()
categorical_columns = df.select_dtypes(exclude=numerics).columns.to_list()

In [5]:
# impute mising numeric features
df_numeric = pd.DataFrame(
    numeric_imputer.transform(df[numeric_columns]), 
    columns=numeric_columns, 
    index=df.index)

# impute mising categorical features
df_categorical = pd.DataFrame(
    categorical_imputer.transform(df[categorical_columns]), 
    columns=categorical_columns, 
    index=df.index)

# concate numeric and categorical features
df = pd.concat([df_numeric, df_categorical], axis=1)

# remove rare labels
df[categorical_columns] = rare_encoder.transform(df[categorical_columns])

# remove outliers
df[numeric_columns] = capper.transform(df[numeric_columns])

# one hot encoding categorical features
df_cat_hotenc = pd.DataFrame(
    enc.transform(df[categorical_columns]), 
    columns=enc.get_feature_names_out(),
    index=df.index) 

# concate numeric and hot-encoded categorical features
df_hotenc = pd.concat([df[numeric_columns], df_cat_hotenc], axis=1)

# predict cluster
labels = model.predict(df_hotenc)

# add cluster label to df
df['cluster'] = labels

df.head()

,Age,Income,Spending_Score,Gender,cluster
CustomerID,,,,,
1,19.0,15.0,39.0,Male,0
2,21.0,15.0,81.0,Male,0
3,20.0,16.0,6.0,Female,0
4,23.0,16.0,77.0,Female,3
5,31.0,17.0,40.0,Female,3


In [6]:
# Cluster Counts
df['cluster'].value_counts()

3    71
0    54
1    53
2    22
Name: cluster, dtype: int64

In [7]:
# Cluster Centers
pd.concat([
    df.groupby(by='cluster').mean(),
    df.groupby(by='cluster').aggregate(pd.Series.mode)[categorical_columns] ],
    axis=1)


C:\Users\Nutzer\AppData\Local\Temp\ipykernel_23472\2379590173.py:4: FutureWarning: ['Age'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df.groupby(by='cluster').aggregate(pd.Series.mode)[categorical_columns] ],


,Age,Income,Spending_Score,Gender
cluster,,,,
0,39.703704,68.759259,27.185185,Male
1,41.226415,59.924528,59.169811,Male
2,32.545455,85.272727,80.590909,Female
3,38.380282,47.140845,51.591549,Female
